In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
# !pip install evaluate
# 
from sklearn.model_selection import train_test_split
import csv
import evaluate

In [3]:
# train_data, test_data = train_test_split(df, test_size=0.3, random_state=42)
# test_data, val_data = train_test_split(test_data, test_size=0.5, random_state=42)
train = pd.read_csv('./datasets/PD_train.csv')
test = pd.read_csv('./datasets/PD_test.csv')
val = pd.read_csv('./datasets/PD_val.csv')

train.reset_index(inplace=True)
test.reset_index(inplace=True)
val.reset_index(inplace=True)

In [4]:
train

,index,Unnamed: 0,CVE_ID,CWE_ID,category,commit_id,commit_message,diff_code,owner,repo,source
0,0,28208,NaN,NaN,1,ce9c22443e77594531be84ba8d523f4148ba09fe,commit ce9c22443e77594531be84ba8d523f4148ba09f...,diff --git a/drivers/media/v4l2-core/videobuf2...,torvalds,linux,wild
1,1,4003,NaN,NaN,0,8ad5db8a8ddbe3bd33078863a027011e28f1f4ee,commit 8ad5db8a8ddbe3bd33078863a027011e28f1f4e...,diff --git a/include/linux/kref.h b/include/li...,torvalds,linux,wild
2,2,10507,NaN,NaN,0,7bdc072086939093238a970f054e8e63d531253d,commit 7bdc072086939093238a970f054e8e63d531253...,diff --git a/drivers/gpu/drm/bridge/analogix/a...,torvalds,linux,wild
3,3,31853,CVE-2012-2794,CWE,1,5ad7335ebac2b38bb2a1c8df51a500b78461c05a,From 5ad7335ebac2b38bb2a1c8df51a500b78461c05a ...,diff --git a/libavcodec/indeo5.c b/libavcodec/...,NaN,NaN,cve
4,4,6625,NaN,NaN,0,2d1c4310774156204c3faff0db583337caeffcc4,commit 2d1c4310774156204c3faff0db583337caeffcc...,diff --git a/drivers/net/usb/cdc_ncm.c b/drive...,stoth68000,media-tree,wild
...,...,...,...,...,...,...,...,...,...,...,...
25065,25065,16850,NaN,NaN,0,28bb6da642cf2a419798b5825597809a1668aacb,commit 28bb6da642cf2a419798b5825597809a1668aac...,diff --git a/epan/dissectors/packet-iscsi.c b/...,wireshark,wireshark,wild
25066,25066,6265,NaN,NaN,0,b5fc571e4f730579f328ae9cf77435cb7fddc53d,commit b5fc571e4f730579f328ae9cf77435cb7fddc53...,diff --git a/libavcodec/aacdec.c b/libavcodec/...,FFmpeg,FFmpeg,wild
25067,25067,11284,NaN,NaN,0,cb4da1a34de3840ce49dc7292a063e1ef7f127af,commit cb4da1a34de3840ce49dc7292a063e1ef7f127a...,diff --git a/drivers/net/wireless/iwlwifi/iwl3...,stoth68000,media-tree,wild
25068,25068,860,NaN,NaN,0,fb8a0ddba29f91af5a2531b12c2e58bbdd69b844,commit fb8a0ddba29f91af5a2531b12c2e58bbdd69b84...,diff --git a/libmpdemux/demux_lavf.c b/libmpde...,mpv-player,mpv,wild


In [5]:
len(train)

25070

In [6]:
train['category'].value_counts()

category
0    16575
1     8495
Name: count, dtype: int64

In [7]:
test['category'].value_counts()

category
0    3537
1    1835
Name: count, dtype: int64

In [8]:
from sklearn import  metrics

# def compute_metrics(y_pred, y_test):
#     # print(y_pred,y_test)
#     # classification_report = metrics.classification_report(y_test,y_pred)
#     accuracy_score = metrics.accuracy_score(y_test,y_pred)
#     recall_score = metrics.recall_score(y_test,y_pred,average='weighted')
#     f1_score = metrics.f1_score(y_test,y_pred,average='weighted')

#     # return {"classification_report": classification_report}
#     return {"accuracy": accuracy_score, "recall": recall_score, "f1":f1_score}

In [9]:
train

,index,Unnamed: 0,CVE_ID,CWE_ID,category,commit_id,commit_message,diff_code,owner,repo,source
0,0,28208,NaN,NaN,1,ce9c22443e77594531be84ba8d523f4148ba09fe,commit ce9c22443e77594531be84ba8d523f4148ba09f...,diff --git a/drivers/media/v4l2-core/videobuf2...,torvalds,linux,wild
1,1,4003,NaN,NaN,0,8ad5db8a8ddbe3bd33078863a027011e28f1f4ee,commit 8ad5db8a8ddbe3bd33078863a027011e28f1f4e...,diff --git a/include/linux/kref.h b/include/li...,torvalds,linux,wild
2,2,10507,NaN,NaN,0,7bdc072086939093238a970f054e8e63d531253d,commit 7bdc072086939093238a970f054e8e63d531253...,diff --git a/drivers/gpu/drm/bridge/analogix/a...,torvalds,linux,wild
3,3,31853,CVE-2012-2794,CWE,1,5ad7335ebac2b38bb2a1c8df51a500b78461c05a,From 5ad7335ebac2b38bb2a1c8df51a500b78461c05a ...,diff --git a/libavcodec/indeo5.c b/libavcodec/...,NaN,NaN,cve
4,4,6625,NaN,NaN,0,2d1c4310774156204c3faff0db583337caeffcc4,commit 2d1c4310774156204c3faff0db583337caeffcc...,diff --git a/drivers/net/usb/cdc_ncm.c b/drive...,stoth68000,media-tree,wild
...,...,...,...,...,...,...,...,...,...,...,...
25065,25065,16850,NaN,NaN,0,28bb6da642cf2a419798b5825597809a1668aacb,commit 28bb6da642cf2a419798b5825597809a1668aac...,diff --git a/epan/dissectors/packet-iscsi.c b/...,wireshark,wireshark,wild
25066,25066,6265,NaN,NaN,0,b5fc571e4f730579f328ae9cf77435cb7fddc53d,commit b5fc571e4f730579f328ae9cf77435cb7fddc53...,diff --git a/libavcodec/aacdec.c b/libavcodec/...,FFmpeg,FFmpeg,wild
25067,25067,11284,NaN,NaN,0,cb4da1a34de3840ce49dc7292a063e1ef7f127af,commit cb4da1a34de3840ce49dc7292a063e1ef7f127a...,diff --git a/drivers/net/wireless/iwlwifi/iwl3...,stoth68000,media-tree,wild
25068,25068,860,NaN,NaN,0,fb8a0ddba29f91af5a2531b12c2e58bbdd69b844,commit fb8a0ddba29f91af5a2531b12c2e58bbdd69b84...,diff --git a/libmpdemux/demux_lavf.c b/libmpde...,mpv-player,mpv,wild


In [10]:
# train['commit_message'][28208]

In [11]:
# id2label = {0: "Adaptive", 1: "POSITIVE"}
# label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [12]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained(
    "./models/codebert-base", num_labels=2
)
tokenizer = AutoTokenizer.from_pretrained('./models/codebert-base')

/root/miniconda3/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at ./models/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/root/miniconda3/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transfor

In [13]:
training_args = TrainingArguments(
    output_dir="./my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

/root/miniconda3/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
encoded_train = tokenizer(train['diff_code'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)
encoded_test = tokenizer(test['diff_code'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)
encoded_val = tokenizer(val['diff_code'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)

In [15]:
class CommitDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [16]:
train_dataset = CommitDataset(encoded_train, list(train['category']))
test_dataset = CommitDataset(encoded_test, list(test['category']))
val_dataset = CommitDataset(encoded_val, list(test['category']))

In [17]:
def compute_metrics(eval_pred):
    accuracy = evaluate.load('./evaluate/metrics/accuracy')
    precision =  evaluate.load("./evaluate/metrics/precision")
    recall =  evaluate.load("./evaluate/metrics/recall")
    f1 =  evaluate.load("./evaluate/metrics/f1")
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy.compute(predictions=predictions, references=labels)['accuracy']
    precision = precision.compute(predictions=predictions, references=labels, average='weighted')['precision']
    recall = recall.compute(predictions=predictions, references=labels,average='weighted')['recall']
    f1 = f1.compute(predictions=predictions, references=labels,average='weighted')['f1']
    # accuracy_score = metrics.accuracy_score(labels,predictions)
    # precision_score = metrics.precision_score(labels,predictions, average='weighted')
    # recall_score = metrics.recall_score(labels,predictions,average='weighted')
    # f1_score = metrics.f1_score(labels,predictions,average='weighted')

    # return {"classification_report": classification_report}
    # return {"precision": precision_score,"recall": recall_score, "f1":f1_score, "accuracy": accuracy_score}
    return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}

In [18]:
# !export https_proxy=http://turbo.gpushare.com:30000 http_proxy=http://turbo.gpushare.com:30000
# !export https_proxy=http://turbo2.gpushare.com:30000 http_proxy=http://turbo2.gpushare.com:30000

In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    # model_init= model_init,
    # data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)
trainer.train()
# best_run = trainer.hyperparameter_search(direction="maximize", hp_space=hp_space, compute_objective=my_compute_objective, n_trials=10)

/tmp/ipykernel_28096/77294352.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.485200,0.467458,{'precision': 0.7792653781244102},{'recall': 0.7842516753536858},{'f1': 0.7791411543312238},{'accuracy': 0.7842516753536858}
2,0.401200,0.458795,{'precision': 0.8003946385478449},{'recall': 0.802680565897245},{'f1': 0.8012027783144016},{'accuracy': 0.802680565897245}
3,0.321900,0.455680,{'precision': 0.8025347370949911},{'recall': 0.8058451228592702},{'f1': 0.8031005069905321},{'accuracy': 0.8058451228592702}
4,0.230100,0.660546,{'precision': 0.8053597212335113},{'recall': 0.8043559195830231},{'f1': 0.8048179520951337},{'accuracy': 0.8043559195830231}
5,0.188300,0.867066,{'precision': 0.8113432520012435},{'recall': 0.8144080416976918},{'f1': 0.8117216717106387},{'accuracy': 0.8144080416976918}
6,0.145500,1.067430,{'precision': 0.8106016420384236},{'recall': 0.8110573343261355},{'f1': 0.8108187605433387},{'accuracy': 0.8110573343261355}


Trainer is attempting to log a value of "{'precision': 0.7792653781244102}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.7842516753536858}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.7791411543312238}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.7842516753536858}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
/tmp/ipykernel_28096/77294352.py:7: UserWarning: To copy construct from a tensor, it is re

TrainOutput(global_step=9402, training_loss=0.30349352247688116, metrics={'train_runtime': 2231.0428, 'train_samples_per_second': 112.369, 'train_steps_per_second': 7.024, 'total_flos': 3.95771649472512e+16, 'train_loss': 0.30349352247688116, 'epoch': 6.0})

In [23]:
fewshot_metrics = trainer.evaluate(eval_dataset=test_dataset)
fewshot_metrics

/tmp/ipykernel_28096/77294352.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Trainer is attempting to log a value of "{'precision': 0.8025347370949911}" of type <class 'dict'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'recall': 0.8058451228592702}" of type <class 'dict'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'f1': 0.8031005069905321}" of type <class 'dict'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'accuracy': 0.8058451228592702}" of type <class 'dict'> for key "eval/accuracy" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.


{'eval_loss': 0.4556801915168762,
 'eval_precision': {'precision': 0.8025347370949911},
 'eval_recall': {'recall': 0.8058451228592702},
 'eval_f1': {'f1': 0.8031005069905321},
 'eval_accuracy': {'accuracy': 0.8058451228592702},
 'eval_runtime': 23.1889,
 'eval_samples_per_second': 231.663,
 'eval_steps_per_second': 14.49,
 'epoch': 6.0}

In [ ]:
# best_run.hyperparameters

In [ ]:
# trainer.apply_hyperparameter(best_run.hyperparameters, final_model=True)
# trainer.train()

In [21]:
1

1

In [22]:
1

1